In [1]:
import pandas as pd

In [2]:
sd = pd.read_csv("data\soil_data.txt")

In [3]:
sd.head

<bound method NDFrame.head of          Year FM               County                 Area   pH  BupH       P  \
0        2001  A                TRIGG   Western Pennyroyal  5.8   6.6    38.0   
1        2001  S  UK SOIL TESTING LAB                  NaN  5.3   6.6    12.0   
2        2001  A                TRIGG   Western Pennyroyal  5.1   6.5    20.0   
3        2001  A           MUHLENBERG  Western Coal Fields  6.8   7.1   175.0   
4        2001  A              HANCOCK  Western Coal Fields  6.2   6.8    88.0   
...       ... ..                  ...                  ...  ...   ...     ...   
1290155  2019  A               MERCER            Bluegrass  6.7   6.9  1091.0   
1290156  2019  A               MERCER            Bluegrass  5.9   6.6   240.0   
1290157  2019  A               MERCER            Bluegrass  6.4   6.8   162.0   
1290158  2019  A                UNION  Western Coal Fields  6.1   6.8   129.0   
1290159  2019  A                UNION  Western Coal Fields  6.0   6.6   281.0  

In [4]:
sd

,Year,FM,County,Area,pH,BupH,P,K,Ca,Mg,Zn,acres,crop
0,2001,A,TRIGG,Western Pennyroyal,5.8,6.6,38.0,177.0,2540.0,161.0,3.2,NaN,Burley Tobacco
1,2001,S,UK SOIL TESTING LAB,NaN,5.3,6.6,12.0,116.0,1730.0,237.0,0.6,NaN,Q/C Sample
2,2001,A,TRIGG,Western Pennyroyal,5.1,6.5,20.0,154.0,1710.0,279.0,1.3,NaN,Fescue
3,2001,A,MUHLENBERG,Western Coal Fields,6.8,7.1,175.0,268.0,3290.0,135.0,2.1,NaN,Corn
4,2001,A,HANCOCK,Western Coal Fields,6.2,6.8,88.0,242.0,3190.0,454.0,1.9,1.0,Burley Tobacco
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290155,2019,A,MERCER,Bluegrass,6.7,6.9,1091.0,1329.0,5510.0,471.0,17.0,NaN,No Info Given
1290156,2019,A,MERCER,Bluegrass,5.9,6.6,240.0,177.0,2772.0,123.0,2.8,NaN,No Info Given
1290157,2019,A,MERCER,Bluegrass,6.4,6.8,162.0,268.0,2928.0,189.0,3.2,NaN,No Info Given
1290158,2019,A,UNION,Western Coal Fields,6.1,6.8,129.0,406.0,2119.0,489.0,4.9,NaN,No Info Given


In [5]:
sd.dtypes

Year        int64
FM         object
County     object
Area       object
pH        float64
BupH      float64
P         float64
K         float64
Ca        float64
Mg        float64
Zn        float64
acres     float64
crop       object
dtype: object

In [6]:
sd.columns

Index(['Year', 'FM', 'County', 'Area', 'pH', 'BupH', 'P', 'K', 'Ca', 'Mg',
       'Zn', 'acres', 'crop'],
      dtype='object')

In [8]:
yrs = sd["Year"]

In [11]:
sd[yrs.isin(yrs[yrs.duplicated()])]

,Year,FM,County,Area,pH,BupH,P,K,Ca,Mg,Zn,acres,crop
0,2001,A,TRIGG,Western Pennyroyal,5.8,6.6,38.0,177.0,2540.0,161.0,3.2,NaN,Burley Tobacco
1,2001,S,UK SOIL TESTING LAB,NaN,5.3,6.6,12.0,116.0,1730.0,237.0,0.6,NaN,Q/C Sample
2,2001,A,TRIGG,Western Pennyroyal,5.1,6.5,20.0,154.0,1710.0,279.0,1.3,NaN,Fescue
3,2001,A,MUHLENBERG,Western Coal Fields,6.8,7.1,175.0,268.0,3290.0,135.0,2.1,NaN,Corn
4,2001,A,HANCOCK,Western Coal Fields,6.2,6.8,88.0,242.0,3190.0,454.0,1.9,1.0,Burley Tobacco
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290155,2019,A,MERCER,Bluegrass,6.7,6.9,1091.0,1329.0,5510.0,471.0,17.0,NaN,No Info Given
1290156,2019,A,MERCER,Bluegrass,5.9,6.6,240.0,177.0,2772.0,123.0,2.8,NaN,No Info Given
1290157,2019,A,MERCER,Bluegrass,6.4,6.8,162.0,268.0,2928.0,189.0,3.2,NaN,No Info Given
1290158,2019,A,UNION,Western Coal Fields,6.1,6.8,129.0,406.0,2119.0,489.0,4.9,NaN,No Info Given
